In [11]:
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
import plotly.graph_objs as go

init_notebook_mode(connected=True)

In [36]:
import sys
# Want to compute throughput when memory is constrained
# Cycles to page in memory: 300
# Cycles to process request (msg sent + received): 500
# L3 cache: 20MB
# memory is per-connection
# 2KB RAM per connection (http://highscalability.com/blog/2013/5/13/the-secret-to-10-million-concurrent-connections-the-kernel-i.html)

def compute_throughput(nconnections, req_rate):
    total_cycles = 2400000000  # 2.4GHz
    total_memory = 20000000  # bytes
    baseline_memory = 2000
    newproto_memory = 200
    
    baseline_cycles = 500
    newproto_cycles = 250
    
    baseline_total_memory = nconnections * baseline_memory
    newproto_total_memory = nconnections * newproto_memory
    
    if baseline_total_memory > total_memory:
        baseline_cycles += 600
    if newproto_total_memory > total_memory:
        newproto_cycles += 600
    
    total_cycles_per_request = total_cycles/req_rate
    print("Total cycles per request: %f" % total_cycles_per_request)
    baseline_cpu = baseline_cycles/total_cycles_per_request
    newproto_cpu = newproto_cycles/total_cycles_per_request
    print("Baseline CPU %%: %f, new protocol CPU %%: %f" % (baseline_cpu, newproto_cpu))
    cycles_improvement = (baseline_cpu - newproto_cpu) * total_cycles
    expected_impact_cycles = cycles_improvement/total_cycles_per_request
    expected_impact_cpu = expected_impact_cycles/req_rate * 100
    return expected_impact_cpu

In [37]:
nconnections = []

x = 100
for i in range(5):
    x = x * 10
    nconnections.append(x)

In [50]:
req_rate = []

x = 1000
for i in range(5):
    x = x * 10
    req_rate.append(x)

In [51]:
req_rate

[10000, 100000, 1000000, 10000000, 100000000]

In [52]:
experiments = []
for i in nconnections:
    for j in req_rate: 
        experiments.append((i, j))

In [53]:
experiments

[(1000, 10000),
 (1000, 100000),
 (1000, 1000000),
 (1000, 10000000),
 (1000, 100000000),
 (10000, 10000),
 (10000, 100000),
 (10000, 1000000),
 (10000, 10000000),
 (10000, 100000000),
 (100000, 10000),
 (100000, 100000),
 (100000, 1000000),
 (100000, 10000000),
 (100000, 100000000),
 (1000000, 10000),
 (1000000, 100000),
 (1000000, 1000000),
 (1000000, 10000000),
 (1000000, 100000000),
 (10000000, 10000),
 (10000000, 100000),
 (10000000, 1000000),
 (10000000, 10000000),
 (10000000, 100000000)]

In [54]:
experiments = [(nconnections, req_rate, compute_throughput(nconnections, req_rate)) for nconnections, req_rate in experiments]

Total cycles per request: 240000.000000
Baseline CPU %: 0.002083, new protocol CPU %: 0.001042
Total cycles per request: 24000.000000
Baseline CPU %: 0.020833, new protocol CPU %: 0.010417
Total cycles per request: 2400.000000
Baseline CPU %: 0.208333, new protocol CPU %: 0.104167
Total cycles per request: 240.000000
Baseline CPU %: 2.083333, new protocol CPU %: 1.041667
Total cycles per request: 24.000000
Baseline CPU %: 20.833333, new protocol CPU %: 10.416667
Total cycles per request: 240000.000000
Baseline CPU %: 0.002083, new protocol CPU %: 0.001042
Total cycles per request: 24000.000000
Baseline CPU %: 0.020833, new protocol CPU %: 0.010417
Total cycles per request: 2400.000000
Baseline CPU %: 0.208333, new protocol CPU %: 0.104167
Total cycles per request: 240.000000
Baseline CPU %: 2.083333, new protocol CPU %: 1.041667
Total cycles per request: 24.000000
Baseline CPU %: 20.833333, new protocol CPU %: 10.416667
Total cycles per request: 240000.000000
Baseline CPU %: 0.004583, 

In [55]:
experiments

[(1000, 10000, 0.10416666666666667),
 (1000, 100000, 1.0416666666666667),
 (1000, 1000000, 10.416666666666668),
 (1000, 10000000, 104.16666666666666),
 (1000, 100000000, 1041.6666666666665),
 (10000, 10000, 0.10416666666666667),
 (10000, 100000, 1.0416666666666667),
 (10000, 1000000, 10.416666666666668),
 (10000, 10000000, 104.16666666666666),
 (10000, 100000000, 1041.6666666666665),
 (100000, 10000, 0.35416666666666663),
 (100000, 100000, 3.5416666666666665),
 (100000, 1000000, 35.416666666666664),
 (100000, 10000000, 354.16666666666663),
 (100000, 100000000, 3541.666666666667),
 (1000000, 10000, 0.10416666666666669),
 (1000000, 100000, 1.0416666666666665),
 (1000000, 1000000, 10.416666666666663),
 (1000000, 10000000, 104.16666666666666),
 (1000000, 100000000, 1041.6666666666672),
 (10000000, 10000, 0.10416666666666669),
 (10000000, 100000, 1.0416666666666665),
 (10000000, 1000000, 10.416666666666663),
 (10000000, 10000000, 104.16666666666666),
 (10000000, 100000000, 1041.666666666667

In [78]:
data = []

for nconnection in nconnections:
    data.append(go.Scatter(x=[req_rate for cxn, req_rate, _ in experiments if cxn == nconnection],
                    y=[pct_change for cxn, _, pct_change in experiments if cxn == nconnection],
                    mode='lines+markers',
                    name=("%dK cxns" % (nconnection / 1000))
                    # marker=dict(
                    # color=[(nconnections) for nconnections, req_rate, _ in experiments if nconnections == nconnection],
                    # colorscale="Viridis",
                    # colorbar=dict(
                    #     title='Cycles per request'
                    # ),
                    # showscale=True,
                    #)
               ))

layout = go.Layout(
    title='Expected percent improvement in application throughput<br>given improvements in per-connection memory utilization',
    xaxis=dict(
        type='log',
        title='Application requests per second (log)'),
    yaxis=dict(
        title='Expected % improvement in app throughput'),
)

fig = go.Figure(data=data, layout=layout)
plot(fig)

'file:///Users/theano/tsch/temp-plot.html'